In [76]:
import pandas as pd
import numpy as np

import gc
import os
import random
import time

import seaborn as sns
from matplotlib import pyplot as plt

import statistics as st
from scipy.stats import skew 
from scipy.stats import kurtosis
from scipy import signal

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split

import xgboost as xgb

In [2]:
SEED = 1729
def determinism(SEED=SEED):
    random.seed = SEED
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed = SEED
    print('done')
    
determinism()

done


In [5]:
gsr=pd.read_csv('/content/gsr_train.csv',header=None)
rr=pd.read_csv('/content/rr_train.csv',header=None)
hr=pd.read_csv('/content/hr_train.csv',header=None)
temp=pd.read_csv('/content/temp_train.csv',header=None)
label=pd.read_csv('/content/labels_train.csv',header=None)

Power Spectrum Calculation

In [6]:
gsr_ps=[]
for i in range(0,632):    
    freqs, psd = signal.welch(gsr.iloc[i])
    gsr_ps.append(np.mean(psd))
rr_ps=[]
for i in range(0,632):    
    freqs, psd = signal.welch(rr.iloc[i])
    rr_ps.append(np.mean(psd))
hr_ps=[]
for i in range(0,632):    
    freqs, psd = signal.welch(hr.iloc[i])
    hr_ps.append(np.mean(psd))

/usr/local/lib/python3.6/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 30, using nperseg = 30
  .format(nperseg, input_length))


In [ ]:
gsr_mean=(gsr.mean(axis=1))
gsr_mean2_gsr=(gsr.mean(axis=1))**2
#gsr_var=(gsr.var(axis=1))
gsr_var=(gsr.sd(axis=1))
gsr_med=gsr.median(axis=1)
gsr_kurt=gsr.kurt(axis=1)
gsr_cs=np.cumsum(gsr,axis=1)[29]
#gsr_ps
hr_mean=(hr.mean(axis=1))
hr_mean2=(hr.mean(axis=1))**2
#hr_var=(hr.var(axis=1))
hr_var=(hr.sd(axis=1))
hr_med=hr.median(axis=1)
hr_kurt=hr.kurt(axis=1)
hr_cs=np.cumsum(hr,axis=1)[29]
#hr_ps

rr_mean=(rr.mean(axis=1))
rr_mean2=(rr.mean(axis=1))**2
#rr_var=(rr.var(axis=1))
rr_var=(rr.sd(axis=1))
rr_med=rr.median(axis=1)
rr_kurt=rr.kurt(axis=1)
rr_cs=np.cumsum(rr,axis=1)[29]
#rr_ps


In [8]:
df=pd.DataFrame(
    list(
        zip(gsr_mean,gsr_var,gsr_med,gsr_kurt,gsr_cs,gsr_ps,
            hr_mean,hr_var,hr_med,hr_kurt,hr_cs,hr_ps,
            rr_mean,rr_var,rr_med,rr_kurt,rr_cs,rr_ps,
            label[0])))



In [9]:
df.columns=['gsr_mean','gsr_var','gsr_med','gsr_kurt','gsr_cs','gsr_ps',
            'hr_mean','hr_var','hr_med','hr_kurt','hr_cs','hr_ps',
            'rr_mean','rr_var','rr_med','rr_kurt','rr_cs','rr_ps',
            'label']


In [10]:
df

,gsr_mean,gsr_var,gsr_med,gsr_kurt,gsr_cs,gsr_ps,hr_mean,hr_var,hr_med,hr_kurt,hr_cs,hr_ps,rr_mean,rr_var,rr_med,rr_kurt,rr_cs,rr_ps,label
0,1.629602,0.000573,1.633575,-1.584146,48.888074,0.000654,62.511111,2.082248,62.000000,1.000413,1875.333333,0.723715,0.942426,0.001626,0.951275,1.669456,28.272768,0.000989,0
1,0.574807,0.015033,0.655739,-0.966358,17.244212,0.030699,74.933333,5.803831,74.666667,-0.338040,2248.000000,13.899084,0.804159,0.003893,0.788120,-0.332033,24.124768,0.006959,1
2,1.020229,0.001051,1.039323,0.185807,30.606870,0.000733,62.500000,1.783525,63.000000,4.560340,1875.000000,0.823004,0.923990,0.056622,0.956805,3.917334,27.719701,0.008351,1
3,1.507063,0.002732,1.504966,-0.961365,45.211902,0.002291,75.466667,1.383908,75.666667,3.515310,2264.000000,1.438966,0.793835,0.000963,0.790885,-0.916222,23.815051,0.001574,0
4,0.378422,0.000007,0.377606,-1.675707,11.352647,0.000014,73.155556,61.285313,76.166667,3.425780,2194.666667,19.293753,0.701381,0.017900,0.745257,2.406221,21.041421,0.008772,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,3.120945,0.000644,3.105660,-0.103505,93.628362,0.000703,84.777778,1.527458,84.333333,-0.977876,2543.333333,3.900179,0.698708,0.014633,0.681655,4.770574,20.961227,0.031894,0
628,2.678209,0.000057,2.677381,-0.727722,80.346274,0.000126,74.555556,7.220945,75.000000,0.865552,2236.666667,16.161541,0.790332,0.004308,0.790885,0.386511,23.709968,0.006209,1
629,1.228856,0.000342,1.228508,-1.158179,36.865673,0.000804,80.933333,3.305747,80.333333,4.022358,2428.000000,2.051780,0.730601,0.001604,0.724517,0.257702,21.918032,0.001117,0
630,0.659290,0.000012,0.659197,-1.183209,19.778713,0.000005,72.077778,3.982248,72.000000,-0.156441,2162.333333,5.004645,0.821304,0.000839,0.824069,0.283480,24.639120,0.001536,0


In [11]:
np.corrcoef(hr_ps,gsr_ps)

array([[ 1.        , -0.03569933],
       [-0.03569933,  1.        ]])

In [69]:
cv = StratifiedKFold(5, shuffle = True)

In [103]:
y = df['label']
X = df.drop(columns = 'label')
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify = y, test_size=0.3)

In [89]:
dict_logi = { 'C': [7.5,8.0,9.0,10.0,12] ,
             'penalty': ['l2','l1'],
             'fit_intercept': [True, False]
}
clf_logi = GridSearchCV(estimator = LogisticRegression(max_iter=1000, random_state=SEED),
                        param_grid = dict_logi,
                        cv = cv,
                        n_jobs=-1
                        )

clf_logi.fit(X_train,y_train)
clf_logi.score(X_train,y_train) , clf_logi.score(X_test, y_test), clf_logi.best_estimator_

(0.620253164556962,
 0.5949367088607594,
 LogisticRegression(C=8.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=1729, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False))

In [104]:
#X_train, X_test, y_train, y_test = train_test_split()
dict_clf = { 'max_depth': [8,15,12,20] ,
             'learning_rate': [0.1,1,2.0],
            'n_estimators': [5,10,100,150],
            'reg_alpha': [0,1,10,100],
            'reg_lambda':[10,100,150]
}
clf = GridSearchCV(estimator = xgb.XGBClassifier(n_jobs=-1, random_state=SEED),
                        param_grid = dict_clf,
                        cv = cv,
                        n_jobs=-1
                        )

clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test), clf.best_estimator_

(0.8778280542986425,
 0.5894736842105263,
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0,
               learning_rate=0.1, max_delta_step=0, max_depth=8,
               min_child_weight=1, missing=None, n_estimators=10, n_jobs=-1,
               nthread=None, objective='binary:logistic', random_state=1729,
               reg_alpha=0, reg_lambda=10, scale_pos_weight=1, seed=None,
               silent=None, subsample=1, verbosity=1))

In [108]:
dict_clf = { 'max_depth': [10,12,18] ,
            'criterion': ['gini', 'entropy'],
            'n_estimators': [100,150,200,250],
             'min_samples_split': [3,5]
}
clf = GridSearchCV(estimator = ExtraTreesClassifier(n_jobs=-1, random_state= SEED, oob_score =True, 
                                                    bootstrap = True),
                        param_grid = dict_clf,
                        cv = cv,
                        n_jobs=-1
                        )

clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test), clf.best_estimator_ 

(0.9140271493212669,
 0.6421052631578947,
 ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                      criterion='entropy', max_depth=10, max_features='auto',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
                      oob_score=True, random_state=1729, verbose=0,
                      warm_start=False))

In [110]:
dict_clf = { 'max_depth': [10,12,18] ,
            'criterion': ['gini', 'entropy'],
            'n_estimators': [200,250],
             'min_samples_split': [5,8]
}
clf = GridSearchCV(estimator = ExtraTreesClassifier(n_jobs=-1, random_state= SEED, oob_score =True, 
                                                    bootstrap = True),
                        param_grid = dict_clf,
                        cv = cv,
                        n_jobs=-1
                        )

clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test), clf.best_estimator_ 

(0.9683257918552036,
 0.6210526315789474,
 ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=12, max_features='auto',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
                      oob_score=True, random_state=1729, verbose=0,
                      warm_start=False))

In [77]:
dict_clf = { 'max_depth': [10,12,18] ,
            'criterion': ['gini', 'entropy'],
            'n_estimators': [100,150,200,250],
             'min_samples_split': [3,5]
}
clf = GridSearchCV(estimator = RandomForestClassifier(n_jobs=-1, random_state= SEED, oob_score =True, 
                                                    bootstrap = True),
                        param_grid = dict_clf,
                        cv = cv,
                        n_jobs=-1
                        )

clf.fit(X_train,y_train)
clf.score(X_train,y_train) , clf.score(X_test, y_test), clf.best_estimator_ 

ValueError: ignored

In [98]:
dict_classifiers = {
    "Logistic Regression": LogisticRegression( C =8.0 ,max_iter=1000 , random_state=SEED),
    "Nearest Neighbors": KNeighborsClassifier(),
    "SVM": SVC(kernel='rbf'),
    "Gradient Boosting Classifier": GradientBoostingClassifier(n_estimators=1000),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=1000),
    "Neural Net": MLPClassifier(alpha = 1),
    "Naive Bayes": GaussianNB(),
    "AdaBoost": AdaBoostClassifier(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "Gaussian Process": GaussianProcessClassifier(),
    "XGBoost": xgb.XGBClassifier(n_jobs=-1, random_state=SEED)
}

In [99]:
def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 5, verbose = True):
    
    dict_models = {}
    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        t_start = time.clock()
        classifier.fit(X_train, Y_train)
        t_end = time.clock()
        
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'train_time': t_diff}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
    return dict_models



def display_dict_models(dict_models, sort_by='test_score'):
    cls = [key for key in dict_models.keys()]
    test_s = [dict_models[key]['test_score'] for key in cls]
    training_s = [dict_models[key]['train_score'] for key in cls]
    training_t = [dict_models[key]['train_time'] for key in cls]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'train_score'] = training_s[ii]
        df_.loc[ii, 'test_score'] = test_s[ii]
        df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))

In [100]:
dict_models = batch_classify(X_train, y_train, X_test, y_test, no_classifiers = 12)
display_dict_models(dict_models)

trained Logistic Regression in 0.20 s
trained Nearest Neighbors in 0.00 s
trained SVM in 0.02 s
trained Gradient Boosting Classifier in 2.84 s
trained Decision Tree in 0.01 s
trained Random Forest in 2.13 s
trained Neural Net in 0.07 s
trained Naive Bayes in 0.00 s
trained AdaBoost in 0.26 s
trained QDA in 0.00 s
trained Gaussian Process in 0.15 s


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


trained XGBoost in 0.22 s


,classifier,train_score,test_score,train_time
0,Logistic Regression,0.620253,0.594937,0.202202
11,XGBoost,0.945148,0.594937,0.220767
4,Decision Tree,1.000000,0.588608,0.007496
3,Gradient Boosting Classifier,1.000000,0.569620,2.844042
5,Random Forest,1.000000,0.569620,2.125011
8,AdaBoost,0.808017,0.569620,0.259321
9,QDA,0.518987,0.563291,0.002771
10,Gaussian Process,1.000000,0.537975,0.154772
1,Nearest Neighbors,0.710970,0.531646,0.002044
6,Neural Net,0.516878,0.531646,0.070775


In [22]:
GDB_params = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.5, 0.1, 0.01, 0.001],
    'criterion': ['friedman_mse', 'mse', 'mae']
}

df_train, df_test, X_train, Y_train, X_test, Y_test = get_train_test(df, y_col, x_cols, 0.6)

for n_est in GDB_params['n_estimators']:
    for lr in GDB_params['learning_rate']:
        for crit in GDB_params['criterion']:
            clf = GradientBoostingClassifier(n_estimators=n_est, 
                                             learning_rate = lr,
                                             criterion = crit)
            clf.fit(X_train, Y_train)
            train_score = clf.score(X_train, Y_train)
            test_score = clf.score(X_test, Y_test)
            print("For ({}, {}, {}) - train, test score: \t {:.5f} \t-\t {:.5f}".format(n_est, lr, crit[:4], train_score, test_score))

For (100, 0.5, frie) - train, test score: 	 1.00000 	-	 0.56917
For (100, 0.5, mse) - train, test score: 	 1.00000 	-	 0.56522
For (100, 0.5, mae) - train, test score: 	 0.74670 	-	 0.56917
For (100, 0.1, frie) - train, test score: 	 0.98945 	-	 0.54941
For (100, 0.1, mse) - train, test score: 	 0.98945 	-	 0.54150
For (100, 0.1, mae) - train, test score: 	 0.73879 	-	 0.56522
For (100, 0.01, frie) - train, test score: 	 0.81003 	-	 0.56126
For (100, 0.01, mse) - train, test score: 	 0.81003 	-	 0.56126
For (100, 0.01, mae) - train, test score: 	 0.74142 	-	 0.56917
For (100, 0.001, frie) - train, test score: 	 0.67810 	-	 0.52964
For (100, 0.001, mse) - train, test score: 	 0.67810 	-	 0.52964
For (100, 0.001, mae) - train, test score: 	 0.72559 	-	 0.56522
For (500, 0.5, frie) - train, test score: 	 1.00000 	-	 0.53755
For (500, 0.5, mse) - train, test score: 	 1.00000 	-	 0.54941
For (500, 0.5, mae) - train, test score: 	 0.74670 	-	 0.56917
For (500, 0.1, frie) - train, test score: